In [10]:
# tf tools
import tensorflow as tf

# image processsing
from tensorflow.keras.preprocessing.image import (load_img,
                                                  img_to_array,
                                                  ImageDataGenerator)
# VGG16 model
from tensorflow.keras.applications.vgg16 import (preprocess_input,
                                                 decode_predictions,
                                                 VGG16)
# cifar10 data - 32x32
from tensorflow.keras.datasets import cifar10

# layers
from tensorflow.keras.layers import (Flatten, 
                                     Dense, 
                                     Dropout, 
                                     BatchNormalization)
# generic model object
from tensorflow.keras.models import Model

# optimizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD

#scikit-learn
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

# for plotting
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import zipfile

## Unzipping file

In [4]:
    folder_path = os.path.join("..", "data", "Test_Alphabet") # Path to the data if unzipped already
    if not os.path.exists(folder_path): # Checking to see if folder is unzipped
        print("Unzipping file")
        path_to_zip = os.path.join("..","data","archive.zip") # Defining the path to the zip file
        zip_destination = os.path.join("..", "data") # defining the output destination

        with zipfile.ZipFile(path_to_zip,"r") as zip_ref: # using the package from zipfile, to un zip the zip file
            zip_ref.extractall(zip_destination)
    print("The files are unzipped")

Unzipping file
The files are unzipped


## Deleting blank folder 

OSError: [Errno 39] Directory not empty: '../data/Test_Alphabet/Blank'

## Image datagenerator 

In [67]:

    print("Creating Image data generator")
    # ImageDataGenerator from tensorflow 
    datagen = ImageDataGenerator(horizontal_flip=True,
                                validation_split = 0.2, # Flip it horizontally around the access randomly 
                                 # Rotate the image randomly 20 degress around the access
                                rescale = 1/255 # rescale the pixel values to between 0-1
                                
    )


Creating Image data generator


## Training images

## Flow from directory

In [68]:
directory = os.path.join("..","data","Train_Alphabet")

In [69]:
training_tensorflow = datagen.flow_from_directory(
    directory,
    target_size=(90, 90),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=None,
    #save_to_dir=None,
    #save_prefix='',
    #save_format='png',
    #follow_links=False,
    subset="training",
    interpolation='nearest',
    keep_aspect_ratio=False
)

Found 18720 images belonging to 26 classes.


## Validation

In [70]:
validation_tensorflow = datagen.flow_from_directory(
    directory,
    target_size=(90, 90),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=None,
    #save_to_dir=None,
    #save_prefix='',
    #save_format='png',
    #follow_links=False,
    subset="validation",
    interpolation='nearest',
    keep_aspect_ratio=False
)

Found 4680 images belonging to 26 classes.


## Test images

In [71]:
    test_datagen = ImageDataGenerator(
                                    rescale = 1./255. # Datagenerator for test, it only has to rescale the images 
    )

In [72]:
directory_test = os.path.join("..","data","Test_Alphabet")

In [73]:
test_tensorflow_ = test_datagen.flow_from_directory(
    directory_test,
    target_size=(90, 90),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=False,
    seed=None,
    #save_to_dir=None,
    #save_prefix='',
    #save_format='png',
    #follow_links=False,
    interpolation='nearest',
    keep_aspect_ratio=False
)

Found 2600 images belonging to 26 classes.


## Loading model

In [74]:

    print("Loading model: ")  
    # load model without classifier layers
    model = VGG16(include_top=False, # Exclude classifier layers
                pooling='avg',
                input_shape=(90, 90, 3)) # Input shape of the images. 224 pixels by 224. 3 color channels

    # Keep pretrained layers, and don't modify them
    for layer in model.layers:
        layer.trainable = False
        
    # Add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    bn = BatchNormalization()(flat1) # Added batnormalization from tensorflow. Take the previouslayer, normalise the values, and than pass them on
    class1 = Dense(256, 
                activation='relu')(bn) # Added new classification layer 
    class2 = Dense(128, 
                activation='relu')(class1) # Added new classification layer with 15 outputs. 15 labels in total
    output = Dense(26, # 15 labels
                activation='softmax')(class2)

    # define new model
    model = Model(inputs=model.inputs, 
                outputs=output)

    # compile
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.01, # Start learning rate at 0.01
        decay_steps=10000, # Every 10 000 steps start decaying 
        decay_rate=0.9) # DEcay by 0.9 to the start learning rate
    sgd = SGD(learning_rate=lr_schedule)

    model.compile(optimizer=sgd,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    # summarize
    print(model.summary())
    

Loading model: 
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 90, 90, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 90, 90, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 90, 90, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 45, 45, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 45, 45, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 45, 45, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 22, 22,

In [75]:
    H = model.fit( # fitting the model to 
        training_tensorflow, # training data from tensorflow dataframe 
        steps_per_epoch = len(training_tensorflow), # Take as many steps as the length of the dataframe 
        validation_data = validation_tensorflow, # Validation data from tensorflow dataframe
        validation_steps = len(validation_tensorflow), # Validation steps as length of validation data 
        epochs = 5)

Epoch 1/5


2023-05-12 13:08:46.837974: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


585/585 [==============================] - ETA: 0s - loss: 2.9458 - accuracy: 0.1683

2023-05-12 13:13:40.272477: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


585/585 [==============================] - 367s 626ms/step - loss: 2.9458 - accuracy: 0.1683 - val_loss: 2.5957 - val_accuracy: 0.2761
Epoch 2/5
306/585 [==============>...............] - ETA: 2:19 - loss: 2.3924 - accuracy: 0.3264

## Predictions 

In [32]:
# Code taken from source in readme file
# Testing the model
print("Testing model with test images")
# Predict the label of the test_images
pred = model.predict(test_tensorflow_) # Using test data on the model
pred = np.argmax(pred,axis=1)

# Map the label 
labels = (training_tensorflow.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]


Testing model with test images


2023-05-12 12:28:17.796338: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


85/85 [==============================] - 42s 493ms/step
['A', 'Y', 'Blank', 'A', 'X', 'A', 'L', 'L', 'Blank', 'Z', 'A', 'A', 'H', 'A', 'A', 'A', 'A', 'L', 'D', 'A', 'A', 'A', 'A', 'A', 'A', 'D', 'A', 'J', 'L', 'A', 'A', 'A', 'A', 'A', 'X', 'N', 'J', 'J', 'F', 'A', 'E', 'A', 'A', 'A', 'A', 'A', 'L', 'H', 'T', 'Z', 'A', 'I', 'A', 'X', 'W', 'N', 'A', 'A', 'A', 'K', 'A', 'A', 'A', 'T', 'G', 'A', 'A', 'P', 'A', 'A', 'A', 'U', 'A', 'T', 'Blank', 'F', 'A', 'A', 'A', 'A', 'L', 'Z', 'A', 'A', 'N', 'A', 'C', 'A', 'I', 'A', 'A', 'M', 'L', 'E', 'A', 'A', 'A', 'Blank', 'A', 'G', 'B', 'B', 'N', 'N', 'B', 'B', 'B', 'L', 'B', 'U', 'A', 'Blank', 'Q', 'I', 'B', 'U', 'Blank', 'B', 'B', 'B', 'L', 'Z', 'B', 'U', 'X', 'I', 'Blank', 'B', 'B', 'Blank', 'B', 'B', 'C', 'Blank', 'M', 'B', 'B', 'L', 'B', 'L', 'B', 'B', 'K', 'B', 'B', 'Z', 'B', 'B', 'B', 'F', 'N', 'Blank', 'U', 'X', 'B', 'U', 'B', 'B', 'U', 'L', 'Blank', 'V', 'Z', 'Blank', 'B', 'Y', 'B', 'B', 'B', 'B', 'I', 'Blank', 'L', 'Blank', 'B', 'B', 'A', 'I

In [ ]:
labels = dict((v,k) for k,v in labels.items())

In [43]:
true_labels = test_tensorflow_.classes

# Convert labels to numerical values
label_map = {v: k for k, v in training_tensorflow.class_indices.items()}
true_labels = np.array([label_map[label] for label in true_labels])

# Print the classification report
print(classification_report(true_labels, pred))


              precision    recall  f1-score   support

           A       0.39      0.55      0.45       100
           B       0.41      0.47      0.44       100
       Blank       0.36      0.74      0.49       100
           C       0.62      0.66      0.64       100
           D       0.30      0.28      0.29       100
           E       0.44      0.26      0.33       100
           F       0.33      0.25      0.28       100
           G       0.52      0.50      0.51       100
           H       0.54      0.64      0.58       100
           I       0.45      0.42      0.44       100
           J       0.28      0.43      0.34       100
           K       0.34      0.20      0.25       100
           L       0.29      0.49      0.37       100
           M       0.29      0.22      0.25       100
           N       0.26      0.26      0.26       100
           O       0.52      0.63      0.57       100
           P       0.35      0.35      0.35       100
           Q       0.20    